In [25]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2)

In [26]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0,
                    3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [27]:
def model(t_u, w, b):
    return w * t_u + b

In [28]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [29]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
params

tensor([1., 0.], requires_grad=True)

In [30]:
params.grad

In [31]:
params.grad is None

True

In [32]:
loss = loss_fn(model(t_u, *params), t_c)
loss

tensor(1763.8846, grad_fn=<MeanBackward0>)

In [33]:
loss.backward()

In [34]:
loss

tensor(1763.8846, grad_fn=<MeanBackward0>)

In [35]:
params

tensor([1., 0.], requires_grad=True)

In [36]:
params.grad

tensor([4517.2969,   82.6000])

In [37]:
params.grad is None

False

In [38]:
if params.grad is not None:
    params.grad.zero_()
params.grad

tensor([0., 0.])

In [39]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:  # <1>
            params.grad.zero_()
        
        t_p = model(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        loss.backward()
        
        with torch.no_grad():  # <2>
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [48]:
training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0, 0.0], requires_grad=True), # <1> 
    t_u = t_un, # <2> 
    t_c = t_c)

Epoch 500, Loss 7.860116
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957697
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

# Explain

In [54]:
n_epochs = 10 
learning_rate = 1e-2 
params = torch.tensor([1.0, 0.0], requires_grad=True) # <1> 
t_u = t_un # <2> 
t_c = t_c
for epoch in range(1, n_epochs + 1):
  if params.grad is not None:  # <1>
      params.grad.zero_()
  
  t_p = model(t_u, *params) 
  loss = loss_fn(t_p, t_c) # (y_pred-y_true) **2 .mean()
  loss.backward()
  print('params.grad: ',params.grad) # contail dw and db (khong can đạo hàm manual nữa)
  with torch.no_grad():  # <2>
      params -= learning_rate * params.grad # update w and b: learning rate * dw, db

  if epoch % 500 == 0:
      print('Epoch %d, Loss %f' % (epoch, float(loss)))
  print('params: {},loss: {}'.format(params,loss))
#print(params)

params.grad:  tensor([-77.6140, -10.6400])
params: tensor([1.7761, 0.1064], requires_grad=True),loss: 80.36434173583984
params.grad:  tensor([-30.8623,  -2.3864])
params: tensor([2.0848, 0.1303], requires_grad=True),loss: 37.57491683959961
params.grad:  tensor([-12.4631,   0.8587])
params: tensor([2.2094, 0.1217], requires_grad=True),loss: 30.871076583862305
params.grad:  tensor([-5.2218,  2.1327])
params: tensor([2.2616, 0.1004], requires_grad=True),loss: 29.756193161010742
params.grad:  tensor([-2.3715,  2.6310])
params: tensor([2.2853, 0.0740], requires_grad=True),loss: 29.50714874267578
params.grad:  tensor([-1.2492,  2.8241])
params: tensor([2.2978, 0.0458], requires_grad=True),loss: 29.392457962036133
params.grad:  tensor([-0.8071,  2.8970])
params: tensor([2.3059, 0.0168], requires_grad=True),loss: 29.298828125
params.grad:  tensor([-0.6325,  2.9227])
params: tensor([ 2.3122, -0.0124], requires_grad=True),loss: 29.208717346191406
params.grad:  tensor([-0.5633,  2.9298])
params: 